<a href="https://colab.research.google.com/github/hevadlli/chicken-disease-detection/blob/main/Chicken_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import kagglehub

In [2]:
dataset_path = kagglehub.dataset_download("allandclive/chicken-disease-1")
print("Dataset location:", dataset_path)
print("Isi folder dataset:", os.listdir(dataset_path))

train_dir = os.path.join(dataset_path, "Train")

100%|██████████| 266M/266M [00:01<00:00, 157MB/s]

Extracting files...


Dataset location: /root/.cache/kagglehub/datasets/allandclive/chicken-disease-1/versions/1
Isi folder dataset: ['Train', 'train_data.csv']


In [3]:
filenames = [f for f in os.listdir(train_dir) if f.endswith(".jpg")]
labels = []

In [4]:
for fname in filenames:
    label_raw = fname.split(".")[0]

    if label_raw.startswith("pcr"):
        label = label_raw[3:]
    else:
        label = label_raw

    labels.append(label)

In [5]:
df = pd.DataFrame({
    "filename": filenames,
    "label": labels
})

In [6]:
print("\nContoh data:")
print(df.head())
print("\nDistribusi kelas:")
print(df["label"].value_counts())

num_classes = df["label"].nunique()


Contoh data:
           filename    label
0    salmo.1175.jpg    salmo
1     salmo.509.jpg    salmo
2    cocci.1143.jpg    cocci
3     healthy.8.jpg  healthy
4  healthy.1299.jpg  healthy

Distribusi kelas:
label
salmo      2625
cocci      2476
healthy    2404
ncd         562
Name: count, dtype: int64


In [7]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [8]:
train_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_dir,
    x_col="filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

Found 6454 validated image filenames belonging to 4 classes.


In [9]:
val_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_dir,
    x_col="filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 1613 validated image filenames belonging to 4 classes.


In [10]:
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

In [12]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,586,948 (9.87 MB)

 Trainable params: 328,964 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [13]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 134s 580ms/step - accuracy: 0.7481 - loss: 0.7026 - val_accuracy: 0.8617 - val_loss: 0.3845
Epoch 2/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 90s 448ms/step - accuracy: 0.8810 - loss: 0.3292 - val_accuracy: 0.9070 - val_loss: 0.2572
Epoch 3/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 90s 445ms/step - accuracy: 0.9034 - loss: 0.2716 - val_accuracy: 0.9107 - val_loss: 0.2415
Epoch 4/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 89s 439ms/step - accuracy: 0.9107 - loss: 0.2494 - val_accuracy: 0.9014 - val_loss: 0.2744
Epoch 5/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 90s 444ms/step - accuracy: 0.9211 - loss: 0.2156 - val_accuracy: 0.9132 - val_loss: 0.2494
Epoch 6/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 89s 442ms/step - accuracy: 0.9199 - loss: 0.2234 - val_accuracy: 0.9175 - val_loss: 0.2425
Epoch 7/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 88s 435ms/step - accuracy: 0.9264 - loss: 0.1997 - val_accuracy: 0.9213 - val_loss: 0.2278
Epoch 8/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 88s 438ms/step - accuracy: 0.9259 - loss: 

In [14]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20
)

Epoch 1/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 176s 627ms/step - accuracy: 0.7906 - loss: 0.6421 - val_accuracy: 0.8890 - val_loss: 0.3760
Epoch 2/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 94s 465ms/step - accuracy: 0.8814 - loss: 0.3391 - val_accuracy: 0.9045 - val_loss: 0.3696
Epoch 3/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 94s 465ms/step - accuracy: 0.9029 - loss: 0.2604 - val_accuracy: 0.9020 - val_loss: 0.3643
Epoch 4/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 93s 462ms/step - accuracy: 0.9136 - loss: 0.2309 - val_accuracy: 0.9082 - val_loss: 0.3337
Epoch 5/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 94s 464ms/step - accuracy: 0.9166 - loss: 0.2106 - val_accuracy: 0.9182 - val_loss: 0.2894
Epoch 6/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 94s 467ms/step - accuracy: 0.9357 - loss: 0.1956 - val_accuracy: 0.9299 - val_loss: 0.2432
Epoch 7/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 93s 461ms/step - accuracy: 0.9450 - loss: 0.1558 - val_accuracy: 0.9237 - val_loss: 0.2573
Epoch 8/20
202/202 ━━━━━━━━━━━━━━━━━━━━ 93s 462ms/step - accuracy: 0.9387 - loss: 